In [ ]:
from utils.configuration import Editor

with Editor('Config') as Config:
    
    Config.Model.Backbone.Name = 'Unet'
    Config.Model.Backbone.Encoder = 'densenet201'
    Config.Model.Head.Name = 'Head_1'
    Config.Model.Head.Param = dict(logit_output=True, in_channels=4)
    
    
    Config.Dataset.ImagesRootPath = '/kaggle/input/cardiac-catheterization/Cardiac_Catheterization/train/images'
    Config.Dataset.BatchSize = 8
    # control input and output image format
    Config.Dataset.IO.InputRGBImage = False
    Config.Dataset.IO.NumWorkers = 0
    Config.Dataset.IO.PinMemory = False
    Config.Dataset.IO.PrefetchFactor = 2
    Config.Dataset.IO.OutputDtype = 'float'
    # uniformed preprocess
    Config.Dataset.Preprocess.Version = 'v1'
    
    

In [ ]:
import Tasks.SimpleSeg.Model.Backbones as Backbones
import Tasks.SimpleSeg.Model.Heads as Heads
from utils import ModelBuilder


def create_model():
    if Config.Model.Backbone.Name == 'Unet':
        backbone_param = dict(encoder_name=Config.Model.Backbone.Encoder, encoder_weights=None, in_channels=1, out_channels=4)

    elif Config.Model.Backbone.Name == 'AttDs':
        assert Config.Model.Backbone.Encoder == None
        backbone_param = dict(in_channels=1, out_channels=4, att_dropout_count=0)


    # start building the model
    backbone_class = getattr(Backbones,Config.Model.Backbone.Name)
    head_class = getattr(Heads,Config.Model.Head.Name)
    model = ModelBuilder(
        backbone_class(**backbone_param),
        head_class(**Config.Model.Head.Param))
    
    model.load()

    model = model.to(device)


In [ ]:

with Editor('Config') as Config:
    # settings for dataset
    Config.Dataset.ImagesRootPath = '/kaggle/input/cardiac-catheterization/Cardiac_Catheterization/train/images'
    Config.Dataset.BatchSize = 8
    # control input and output image format
    Config.Dataset.IO.InputRGBImage = False
    Config.Dataset.IO.NumWorkers = 0
    Config.Dataset.IO.PinMemory = False
    Config.Dataset.IO.PrefetchFactor = 2
    Config.Dataset.IO.OutputDtype = 'float'
    # uniformed preprocess
    Config.Dataset.Preprocess.Version = 'v1'
    
    #choose training structure, including the model, loss, metrics, optimizer, schedular
    Config.Task = 'SimpleSeg'
    # model
    Config.Model.Backbone.Name = BackboneName
    Config.Model.Backbone.Param = {}
    Config.Model.Head.Name = 'Head_1'
    Config.Model.Head.Param = dict(logit_output=True, in_channels=4)
    # loss
    Config.Training.Loss.Name = 'DiceBCELoss'
    Config.Training.Loss.Param = dict(use_logit=True,w_bce=0.2)
    # optimizer
    Config.Training.Optimizer.Name = 'Adam'
    Config.Training.Optimizer.Param = dict(lr=0.001)
    # scheduler
    Config.Training.Scheduler.Name = 'Custom_1'
    Config.Training.Scheduler.Param = dict(warmup_epochs=1,reduce_gamma=-2)
    # metrics
    Config.Training.Metrics.Name = ['DiceBCELoss','DiceAccuracy']
    Config.Training.Metrics.Param = [dict(use_logit=True,w_bce=0.2),dict(use_logit=True,threshold=0.5)]
    
    # whether to freeze backbone
    Config.Training.Settings.Model.FreezeBackbone = False
    
    # setup training process
    Config.Training.Settings.Epochs = 30
    Config.Training.Settings.GradientAccumulation = 1
    Config.Training.Settings.AmpScaleTrain = True
    # set random property
    Config.Training.Settings.Random.cuDNN.Deterministic = True
    Config.Training.Settings.Random.cuDNN.Benchmark = True
    Config.Training.Settings.Random.Seed.Dataset.Split = SplitSeed
    Config.Training.Settings.Random.Seed.Dataset.Transform = TransformSeed
    Config.Training.Settings.Random.Seed.Dataset.Shuffle = ShuffleSeed
    Config.Training.Settings.Random.Seed.Model = ModelSeed
    
    Config.Logging.StepsPerLog = 8
    Config.Logging.Image.Columns = 3
    Config.Logging.Image.Rows = 10
    Config.Logging.Image.Figsize = (300,300)
    Config.Logging.Image.Fontsize = 200
    Config.Logging.Image.DPI = 10
    Config.Logging.Image.MaskAlpha = 0.4
    Config.Logging.Image.Threshold = 0.5
    
    Config.Logging.RootPath = '/kaggle/working'
    Config.Logging.FolderName = FolderName
    Config.Logging.TrialNumber = TrialNumber
    Config.Logging.Note = ''

In [ ]:
import io
import pathlib
import random
import math
import numpy as np
import matplotlib.pyplot as plt

import torch
from torch.cuda.amp import autocast, GradScaler


from Dataset import TrainDataLoader

import Transform.Preprocess
import Transform.Combinations

import Schedule.LearningRate

import Tasks

from utils import Recorder
from utils import ModelBuilder

import utils.configuration
Config = utils.configuration.Config


# Select the task and use correspondent model, loss_fn, metrics
task = getattr(Tasks,Config.Training.Task)
# seed for model creation
model_seed = Config.Training.Settings.Random.Seed.Model
torch.manual_seed(model_seed)
# start building the model
# build the model backbone
model_backbone_class = getattr(task.Model.Backbones,Config.Training.Model.Backbone.Name)
model_backbone_param = Config.Training.Model.Backbone.Param
model_backbone = model_backbone_class(**model_backbone_param)
# build the model head
model_head_class = getattr(task.Model.Heads,Config.Training.Model.Head.Name)
model_head_param = Config.Training.Model.Head.Param
model_head = model_head_class(**model_head_param)
# create the model
model = ModelBuilder(model_backbone,model_head)
model = model.to(device)
